### Video Pre-processor

Consumes the videos from a file and produces files that can be used by downstream models and/or downstream preprocessors. 

This is the first step in the video preprocessing pipeline.

In [0]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Create a tiny video folder for testing and debugging purposes

In [0]:
!cp 'drive/My Drive/cs231n-project/datasets/emotiw/val_frames.tar.gz' .
!tar -xzf val_frames.tar.gz

In [0]:
!ls

drive	  openpose.tar.gz  val_frames	      val-tiny-pose_tmp
openpose  sample_data	   val_frames.tar.gz


In [0]:
import os
videos = next(os.walk("drive/My Drive/cs231n-project/datasets/emotiw/val"))[2]
print(len(videos))



766


In [0]:
!mkdir test-tiny

In [0]:
from shutil import copyfile

i = len(videos) - 1
j = 0
while j < 50:
    copyfile(os.path.join("drive/My Drive/cs231n-project/datasets/emotiw/val", videos[i]), os.path.join("test-tiny", videos[i]))
    j += 1
    i -= 1

In [0]:
!ls test-tiny

100_1.mp4  44_29.mp4  5_2.mp4	 53_3.mp4   54_13.mp4  81_13.mp4  86_42.mp4
11_12.mp4  44_9.mp4   53_12.mp4  53_47.mp4  54_8.mp4   81_4.mp4   86_6.mp4
11_1.mp4   45_1.mp4   53_14.mp4  53_48.mp4  55_19.mp4  83_1.mp4
31_2.mp4   45_5.mp4   53_15.mp4  53_49.mp4  55_2.mp4   86_11.mp4
31_3.mp4   45_6.mp4   53_33.mp4  53_7.mp4   55_7.mp4   86_19.mp4
31_5.mp4   5_11.mp4   53_35.mp4  53_8.mp4   55_8.mp4   86_20.mp4
44_17.mp4  5_16.mp4   53_36.mp4  53_9.mp4   70_24.mp4  86_26.mp4
44_26.mp4  5_1.mp4    53_39.mp4  54_10.mp4  77_3.mp4   86_40.mp4


In [0]:
!zip -j test-tiny.zip test-tiny/*

In [0]:
!cp test-tiny.zip "drive/My Drive/cs231n-project/datasets/emotiw"


### Video preprocessing to extract frames

In [0]:
import zipfile
import cv2
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os
from os.path import isfile, join

class VideoPreprocessor:
    """
    Preprocesses raw videos into frames so that further downstream extraction and
    preprocessing can occur

    Example usage:

    video_preprocessor = VideoPreprocessor(
        video_folder="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny.zip", 
        label_file="drive/My Drive/cs231n-project/datasets/emotiw/Train_labels.txt", 
        output_folder="train-tiny-local", 
        output_file="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-local.zip"
    )
    video_preprocessor.preprocess()

    """

    def __init__(self, video_folder, output_folder, output_file=None, label_file=None, is_zip=True, height=320, width=480, sample_every=10, max_workers=32):
        """
        @param video_folder   The folder where the list of videos are stored. If 
                              `is_zip` is set to True, this should be a single zip 
                              file containing the videos. Paths can either by a local 
                              folder or a GDrive mounted path.
        @param label_file     The file containing the space-delimited video name to label mapping. If None,
                              we assume that we are in 'test' mode and that there are no categories
        @param output_folder  The local output path where the preprocessed files will be stored for 
                              further preprocessing can be done
        @param output_file    If not none, the output_folder will be zipped up and stored at this location
        @param is_zip         If set to True, the `video_folder` will be unzipped prior to accessing
        @param height         Height of the extracted video frames
        @param width          Width of the extracted video frames
        @param sample_every   The frames to skip.
        @param max_workers    The number of workers to use to parallelize work.
        """
        self.is_zip = is_zip
        self.video_folder = video_folder
        self.label_file = label_file
        self.output_folder = output_folder
        self.output_file = output_file
        print(f"Video Preprocessor created with is_zip = {is_zip}, video_folder = {video_folder} , label_file = {label_file} , output_folder = {output_folder}, output_file = {output_file}")
        
        self.height = height
        self.width = width
        self.sample_every = sample_every
        self.max_workers = max_workers
        print(f"Frames will be created with height = {height} , width = {width} , sample_every = {sample_every}")


    def preprocess(self):
        if self.is_zip:
            # Unzips files to a temp directory
            tmp_output_folder = self.output_folder.rstrip('/') + "_tmp"
            print(f"Unzipping files to temp dir {tmp_output_folder}...")
            Path(f"{tmp_output_folder}").mkdir(parents=True, exist_ok=True)
            with zipfile.ZipFile(self.video_folder, 'r') as zip_ref:
                zip_ref.extractall(tmp_output_folder)
            print("Finished unzipping files")
        else:
            tmp_output_folder = self.video_folder
            print("Skipping unzipping files as input is a folder")
        
        if self.label_file is not None:
            # Create the category subfolders in the output folder
            # Path Structure:
            #   output/
            #     1/
            #     2/
            video_to_label = {}
            unique_labels = set()
            with open(self.label_file, "r") as f:
                i = 0
                for line in f:
                    if i == 0:
                        i += 1
                        continue
                    line_arr = line.split(" ")
                    video_to_label[line_arr[0] + ".mp4"] = line_arr[1].strip()
                    if line_arr[1].strip() not in unique_labels:
                        unique_labels.add(line_arr[1].strip())
                        Path(f"{self.output_folder}/{line_arr[1].strip()}").mkdir(parents=True, exist_ok=True)
                    i += 1
        else:
            video_to_label = None
            Path(f"{self.output_folder}").mkdir(parents=True, exist_ok=True)

        # Process each video by extracting the frames in a multi-threaded fashion
        futures = []
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            videos = next(os.walk(tmp_output_folder))[2]
            videos = sorted(videos)

            print(f"Found {len(videos)} videos")
            video_num = 1

            for video_name in videos:
                future = executor.submit(self.process_video, tmp_output_folder, video_name, video_num, len(videos), video_to_label)
                futures.append(future)
                video_num += 1

        cv2.destroyAllWindows()

        print("***** Submitted all tasks *****")
        for future in futures:
            future.result()
        print("***** Completed *****")

        if self.output_file is not None:
            print(f"Starting to zip files to {self.output_file}")
            def zipdir(path, ziph):
                for root, dirs, files in os.walk(path):
                    folder = root[len(path):]
                    for file in files:
                        ziph.write(join(root, file), join(folder, file))

            zipf = zipfile.ZipFile(self.output_file, 'w', zipfile.ZIP_DEFLATED)
            zipdir(self.output_folder, zipf)
            zipf.close()
            print(f"Done zipping files to {self.output_file}")
        
        print("Done!")

    def process_video(self, tmp_output_folder, video_name, video_num, total_videos, video_to_label):
        """
        Processes a video by extracting the frames
        Writes out each frame, where each frame is an image resized to the the specified dimensions
        """
        vidcap = cv2.VideoCapture(join(tmp_output_folder, video_name))
        if video_to_label is not None:
            label = video_to_label[video_name]
            print(f"Processing video {video_num}/{total_videos} with name {video_name} and class {label} \n")
        else:
            label = None
            print(f"Processing video {video_num}/{total_videos} with name {video_name} (test-mode) \n")

        input_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(vidcap.get(cv2.CAP_PROP_FPS))

        success, image = vidcap.read()
        count = 0
        frame = 0
        while success:
            if count % self.sample_every == 0:
                height, width = image.shape[:2]
                image = cv2.resize(image, (self.width, self.height), interpolation = cv2.INTER_CUBIC)
                if video_to_label is not None:
                    cv2.imwrite(f"{self.output_folder}/{label}/frame_{video_name}_{frame}.jpg", image)
                else:
                    cv2.imwrite(f"{self.output_folder}/frame_{video_name}_{frame}.jpg", image)

                frame += 1
            success, image = vidcap.read()
            count += 1
        video_num += 1
        vidcap.release()





In [0]:
HOME_DIR = "drive/My Drive/Machine-Learning-Projects/"
video_preprocessor = VideoPreprocessor(
    video_folder= HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny.zip", 
    label_file= HOME_DIR + "cs231n-project/datasets/emotiw/Train_labels.txt", 
    output_folder="train-tiny-local", 
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny-local.zip"
)
video_preprocessor.preprocess()

In [0]:
HOME_DIR = "drive/My Drive/"
video_preprocessor = VideoPreprocessor(
    video_folder= HOME_DIR + "cs231n-project/datasets/emotiw/val-tiny.zip", 
    label_file= HOME_DIR + "cs231n-project/datasets/emotiw/Val_labels.txt", 
    output_folder="val-tiny-local", 
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/val-tiny-local.zip"
)
video_preprocessor.preprocess()

In [0]:
HOME_DIR = "drive/My Drive/"
video_preprocessor = VideoPreprocessor(
    video_folder= HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny.zip", 
    output_folder="test-tiny-local", 
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny-local.zip"
)
video_preprocessor.preprocess()

Video Preprocessor created with is_zip = True, video_folder = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny.zip , label_file = None , output_folder = test-tiny-local, output_file = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-local.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Unzipping files to temp dir test-tiny-local_tmp...
Finished unzipping files
Found 50 videos
Processing video 1/50 with name 100_1.mp4 (test-mode) 

Processing video 2/50 with name 11_1.mp4 (test-mode) 

Processing video 4/50 with name 31_2.mp4 (test-mode) 

Processing video 6/50 with name 31_5.mp4 (test-mode) 

Processing video 8/50 with name 44_26.mp4 (test-mode) 

Processing video 9/50 with name 44_29.mp4 (test-mode) 

Processing video 11/50 with name 45_1.mp4 (test-mode) 

Processing video 12/50 with name 45_5.mp4 (test-mode) 

Processing video 15/50 with name 53_14.mp4 (test-mode) 
Processing video 14/50 with name 53_12.mp4 (test-mode) 


Processing video 

In [0]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 41kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=b390a13e5824c6e7568babbe9c2f2c7809afc357698b622c130927e96f469dae
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
import zipfile
import cv2
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os
from os.path import isfile, join
import face_recognition
import pickle

class FacePreprocessor:
    """
    Extract the faces from the videos.
    Faces are stored in a flat directory structure (no categorical hierarchy)
    
    Processes a video by extracting the faces from each frame and creating a 
    list of list of faces which is then saved as a pickled object.

    NOTE: Faces are not guaranteed to be the same across frames.
          eg. 'face 1' in frame 1 may not be the same as 'face 1' in frame 2

    Pickle Object Format:
        [
            [
                [frame 1, face 1],
                [frame 1, face 2],
                [frame 1, face 3]
            ],
            [
                [frame 2, face 1],
                [frame 2, face 2]
            ],
            ...
        ]

    """

    def __init__(self, video_folder, output_folder, output_file=None, is_zip=True, height=320, width=480, sample_every=10, max_workers=32):
        """
        @param video_folder          The folder where the list of videos frames are stored. If 
                                     `is_zip` is set to True, this should be a single zip 
                                     file containing the video frames. Paths can either by a local 
                                     folder or a GDrive mounted path.
        @param output_folder         The local output path where the preprocessed files will be stored for 
                                     further preprocessing can be done
        @param output_file           If not none, the output_folder will be zipped up and stored at this location
        @param is_zip                If set to True, the `video_folder` will be unzipped prior to accessing
        @param height         Height of the extracted video frames
        @param width          Width of the extracted video frames
        @param sample_every   The frames to skip.
        @param max_workers    The number of workers to use to parallelize work.
        """
        self.is_zip = is_zip
        self.video_folder = video_folder
        self.output_folder = output_folder
        self.output_file = output_file
        print(f"Video Preprocessor created with is_zip = {is_zip}, video_folder = {video_folder} , output_folder = {output_folder}, output_file = {output_file}")

        self.height = height
        self.width = width
        self.sample_every = sample_every
        self.max_workers = max_workers
        print(f"Frames will be created with height = {height} , width = {width} , sample_every = {sample_every}")

    def preprocess(self):
        tmp_output_folder = ""
        if self.is_zip:
            # Unzips files to a temp directory
            tmp_output_folder = self.output_folder.rstrip('/') + "_tmp"
            print(f"Unzipping files to temp dir {tmp_output_folder}...")
            Path(f"{tmp_output_folder}").mkdir(parents=True, exist_ok=True)
            with zipfile.ZipFile(self.video_folder, 'r') as zip_ref:
                zip_ref.extractall(tmp_output_folder)
            print("Finished unzipping files")
        else:
            tmp_output_folder = self.video_folder
            print("Skipping unzipping files as input is a folder")

        # Create output folder
        Path(f"{self.output_folder}/faces-pickle/").mkdir(parents=True, exist_ok=True)

        # Process each video by extracting the frames in a multi-threaded fashion
        futures = []
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            videos = next(os.walk(tmp_output_folder))[2]
            print(f"Found {len(videos)} videos")
            video_num = 1

            for video_name in videos:
                future = executor.submit(self.process_video, self.process_audio m tmp_output_folder, video_name, video_num, len(videos))
                futures.append(future)
                video_num += 1

        cv2.destroyAllWindows()

        print("***** Submitted all tasks *****")
        with open(f"{self.output_folder}/summary.txt", 'w') as f:
            f.write(f"video_name,face_image_name,frame_number,face_number,total_frames,fps,video_width,video_height,top,right,bottom,left\n")
            for future in futures:
                out_arr = future.result()
                for out in out_arr:
                    f.write(out)
        print("***** Completed *****")


        if self.output_file is not None:
            print(f"Starting to zip files to {self.output_file}")
            def zipdir(path, ziph):
                for root, dirs, files in os.walk(path):
                    folder = root[len(path):]
                    for file in files:
                        ziph.write(join(root, file), join(folder, file))

            zipf = zipfile.ZipFile(self.output_file, 'w', zipfile.ZIP_DEFLATED)
            zipdir(self.output_folder, zipf)
            zipf.close()
            print(f"Done zipping files to {self.output_file}")
        
        print("Done!")


    def process_video(self, tmp_output_folder, video_name, video_num, total_videos):
        """
        Processes a video by extracting the faces
        """
        vidcap = cv2.VideoCapture(join(tmp_output_folder, video_name))
        print(f"Processing video {video_num}/{total_videos} with name {video_name} \n")

        input_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(vidcap.get(cv2.CAP_PROP_FPS))

        metadata = []
        faces_all_frames = []
        success, image = vidcap.read()
        count = 0
        frame = 0
        while success:
            if count % self.sample_every == 0:
                height, width = image.shape[:2]
                image = cv2.resize(image, (self.width, self.height), interpolation = cv2.INTER_CUBIC)

                # Convert from BGR color (OpenCV) to RGB color (face_recognition)
                rgb_image = image[:, :, ::-1]

                # Find all the faces in the current frame of video
                face_locations = face_recognition.face_locations(rgb_image)
                faces = []
                face_num = 0
                # Display the results
                for top, right, bottom, left in face_locations:
                    # Draw a box around the face
                    faces.append(image[top:bottom, left:right, :].copy())
                    metadata.append(f"{video_name},frame-{count}.face-{face_num}.jpg,{count},{face_num},{input_length},{fps},{frame_width},{frame_height},{top},{right},{bottom},{left}\n")
                    face_num += 1
                faces_all_frames.append(faces)

                frame += 1
            success, image = vidcap.read()
            count += 1
        video_num += 1
        vidcap.release()

        with open(f"{self.output_folder}/faces-pickle/{video_name}.pkl", "wb") as f_out:
            pickle.dump(faces_all_frames, f_out)
        return metadata





In [0]:
face_preprocessor = FacePreprocessor(
    video_folder="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny.zip",
    output_folder="train-tiny-faces", 
    output_file="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-faces.zip"
)
face_preprocessor.preprocess()

Video Preprocessor created with is_zip = True, video_folder = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny.zip , output_folder = train-tiny-faces, output_file = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Unzipping files to temp dir train-tiny-faces_tmp...
Finished unzipping files
Found 50 videos
Processing video 1/50 with name 324_96.mp4 

Processing video 14/50 with name 112_5.mp4 

Processing video 5/50 with name 2_2.mp4 
Processing video 3/50 with name 34_9.mp4 


Processing video 13/50 with name 188_22.mp4 

Processing video 11/50 with name 276_3.mp4 

Processing video 16/50 with name 16_14.mp4 

Processing video 7/50 with name 33_20.mp4 

Processing video 4/50 with name 204_13.mp4 

Processing video 15/50 with name 69_30.mp4 
Processing video 8/50 with name 97_22.mp4 
Processing video 17/50 with name 334_21.mp4 
Processing video 9/50 with name 277_3.mp4 

Processing video

### Pose Extraction

Poses must be extracted from the frames so VideoProcessor must be run before this.

In [0]:
!cp drive/'My Drive'/cs231n-project/openpose/openpose.tar.gz .

In [0]:
!tar -xzf openpose.tar.gz

In [0]:
!ls openpose

3rdparty      build  CMakeLists.txt  examples  LICENSE	python	   scripts
appveyor.yml  cmake  doc	     include   models	README.md  src


In [0]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

Selecting previously unselected package libgflags2.2.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [0]:
import zipfile
import cv2
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os
from os.path import isfile, join
import pickle
import subprocess
import os

class PosePreprocessor:
    """
    Extract the poses from the video frames.
    """

    def __init__(self, video_frame_folder, output_folder, output_file=None, is_zip=True, is_test=False):
        """
        @param video_frame_folder    The folder where the list of videos frames are stored. If 
                                     `is_zip` is set to True, this should be a single zip 
                                     file containing the video frames. Paths can either by a local 
                                     folder or a GDrive mounted path.
        @param output_folder         The local output path where the preprocessed files will be stored for 
                                     further preprocessing can be done
        @param output_file           If not none, the output_folder will be zipped up and stored at this location
        @param is_zip                If set to True, the `video_frame_folder` will be unzipped prior to accessing
        @param is_test               If set to True, the `video_frame_folder` is assumed to have no categorical 
                                     classification folder hierarchy
        """
        self.is_zip = is_zip
        self.is_test = is_test
        self.video_frame_folder = video_frame_folder
        self.output_folder = output_folder
        self.output_file = output_file
        print(f"Pose Preprocessor created with is_zip = {is_zip}, is_test = {is_test}, video_frame_folder = {video_frame_folder} , output_folder = {output_folder}, output_file = {output_file}")

    def preprocess(self):
        tmp_output_folder = ""
        if self.is_zip:
            # Unzips files to a temp directory
            tmp_output_folder = self.output_folder.rstrip('/') + "_tmp"
            print(f"Unzipping files to temp dir {tmp_output_folder}...")
            Path(f"{tmp_output_folder}").mkdir(parents=True, exist_ok=True)
            with zipfile.ZipFile(self.video_frame_folder, 'r') as zip_ref:
                zip_ref.extractall(tmp_output_folder)
            print("Finished unzipping files")
        else:
            tmp_output_folder = self.video_frame_folder
            print("Skipping unzipping files as input is a folder")

        # Create output folder for the keypoints
        Path(f"{self.output_folder}").mkdir(parents=True, exist_ok=True)

        if self.is_test:
            print(f"Starting pose extraction for {tmp_output_folder}")
            p = subprocess.run(["build/examples/openpose/openpose.bin", "--image_dir", "../" + tmp_output_folder, "--write_json", "../" + self.output_folder, "--display", "0", "--render_pose", "0"], cwd="openpose", stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8')
            print(p.stdout)
            print(p.stderr)
        else:
            # Subfolders represent the different categories 
            # (we will mimic this for the final output)
            subfolders = next(os.walk(tmp_output_folder))[1]
            for subfolder in subfolders:
                print(f"Starting pose extraction for {join(tmp_output_folder, subfolder)}")
                p = subprocess.run(["build/examples/openpose/openpose.bin", "--image_dir", "../" + join(tmp_output_folder, subfolder), "--write_json", "../" + join(self.output_folder, subfolder), "--display", "0", "--render_pose", "0"], cwd="openpose", stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8')
                print(p.stdout)
                print(p.stderr)

        if self.output_file is not None:
            print(f"Starting to zip files to {self.output_file}")
            def zipdir(path, ziph):
                for root, dirs, files in os.walk(path):
                    folder = root[len(path):]
                    for file in files:
                        ziph.write(join(root, file), join(folder, file))

            zipf = zipfile.ZipFile(self.output_file, 'w', zipfile.ZIP_DEFLATED)
            zipdir(self.output_folder, zipf)
            zipf.close()
            print(f"Done zipping files to {self.output_file}")
        
        print("Done!")






In [0]:
pose_preprocessor = PosePreprocessor(
    video_frame_folder="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-local.zip",
    output_folder="train-tiny-pose", 
    output_file="drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-pose.zip"
)
pose_preprocessor.preprocess()

Pose Preprocessor created with is_zip = True, is_test = False, video_frame_folder = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-local.zip , output_folder = train-tiny-pose, output_file = drive/My Drive/cs231n-project/datasets/emotiw/train-tiny-pose.zip
Unzipping files to temp dir train-tiny-pose_tmp...
Finished unzipping files
Starting pose extraction for train-tiny-pose_tmp/1
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 14.118342 seconds.


Starting pose extraction for train-tiny-pose_tmp/3
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 14.530537 seconds.


Starting pose extraction for train-tiny-pose_tmp/2
Starting OpenPose demo...
Configuring O

In [0]:
pose_preprocessor = PosePreprocessor(
    video_frame_folder="drive/My Drive/cs231n-project/datasets/emotiw/val-tiny-local.zip",
    output_folder="val-tiny-pose", 
    output_file="drive/My Drive/cs231n-project/datasets/emotiw/val-tiny-pose.zip"
)
pose_preprocessor.preprocess()

Pose Preprocessor created with is_zip = True, is_test = False, video_frame_folder = drive/My Drive/cs231n-project/datasets/emotiw/val-tiny-local.zip , output_folder = val-tiny-pose, output_file = drive/My Drive/cs231n-project/datasets/emotiw/val-tiny-pose.zip
Unzipping files to temp dir val-tiny-pose_tmp...
Finished unzipping files
Starting pose extraction for val-tiny-pose_tmp/1
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 15.779708 seconds.


Starting pose extraction for val-tiny-pose_tmp/3
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 8.034669 seconds.


Starting pose extraction for val-tiny-pose_tmp/2
Starting OpenPose demo...
Configuring OpenPose...
Star

In [0]:
pose_preprocessor = PosePreprocessor(
    video_frame_folder="drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-local.zip",
    output_folder="test-tiny-pose", 
    output_file="drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-pose.zip",
    is_test=True
)
pose_preprocessor.preprocess()

Pose Preprocessor created with is_zip = True, is_test = True, video_frame_folder = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-local.zip , output_folder = test-tiny-pose, output_file = drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-pose.zip
Unzipping files to temp dir test-tiny-pose_tmp...
Finished unzipping files
Starting pose extraction for test-tiny-pose_tmp
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 34.017663 seconds.


Starting to zip files to drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-pose.zip
Done zipping files to drive/My Drive/cs231n-project/datasets/emotiw/test-tiny-pose.zip
Done!


## Audio Preprocessor

In [0]:
import zipfile
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import os
%tensorflow_version 1
import tensorflow as tf
from os.path import isfile, join
import pickle
import glob
import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
import numpy as np
subprocess.check_output("pip install openl3" , shell=True)
subprocess.check_output("pip install pydub", shell=True)
import openl3
import soundfile as sf
class AudioPreprocessor:
    """
    Extract the audio from the videos.
    Faces are stored in a flat directory structure (no categorical hierarchy)
    

    """

    def __init__(self, video_folder, output_folder, output_file=None, label_path = None , is_zip=True, sample_every=10 , hop_size=0.5, max_len=5):
        """
        @param video_folder          The folder where the list of videos frames are stored. If 
                                     `is_zip` is set to True, this should be a single zip 
                                     file containing the video frames. Paths can either by a local 
                                     folder or a GDrive mounted path.
        @param output_folder         The local output path where the preprocessed files will be stored for 
                                     further preprocessing can be done
        @param output_file           If not none, the output_folder will be zipped up and stored at this location
        @param label_path            The path of the .txt file containing the class labels matched to the sample name.
        @param is_zip                If set to True, the `video_folder` will be unzipped prior to accessing
        - hop_size: The frame collection rate 
        @param sample_every the frames to skip.

        """
        self.is_zip = is_zip
        self.video_folder = video_folder
        self.output_folder = output_folder
        self.output_file = output_file
        self.hop_size = hop_size
        self.max_len = max_len
        self.label_path = label_path
        print(f"Video Preprocessor created with is_zip = {is_zip}, video_folder = {video_folder} , output_folder = {output_folder}, output_file = {output_file}")

        self.sample_every = sample_every
        print(f"Frames will be created with hop_size = {hop_size}")


    def preprocess(self, batch_size=16):
      """
      Outputs: Writes to disk the openl3 embedding pickle object for each sample. 
      Optionally, it will output the entire matched X and Y numpy pickle objects if label path is provided
      - 
      """


      tmp_output_folder = ""
      if self.is_zip:
      # Unzips files to a temp directory
        tmp_output_folder = self.output_folder.rstrip('/') + "_tmp"
        print(f"Unzipping files to temp dir {tmp_output_folder}...")
        Path(f"{tmp_output_folder}").mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(self.video_folder, 'r') as zip_ref:
            zip_ref.extractall(tmp_output_folder)
        print("Finished unzipping files")
      else:
        tmp_output_folder = self.video_folder
        print("Skipping unzipping files as input is a folder")

      Path(f"{self.output_folder}/audio-pickle/").mkdir(parents=True, exist_ok=True)

      # Strip the audio from video and store as .wav file
      video_files = sorted(glob.glob(tmp_output_folder + '/*.mp4'))
      video_files_split = np.array_split(np.asarray(video_files),len(video_files)//batch_size)

      target_labels = []

      if self.label_path is not None:   
        targets = []
        target_labels = np.genfromtxt(self.label_path , delimiter = ' ' , dtype='str')

      sr = 0
      all_x = []

      maxlen = int(self.max_len // self.hop_size + 1)

      for i in range (0 , len(video_files_split)):


        audio_reads = []


        for f in video_files_split[i]: 
          newname = os.path.basename(f)
          output_wav_file = newname + 'extracted_audio.wav'
          ffmpeg_extract_audio(f ,  "/tmp/" + output_wav_file)
          if self.label_path is not None:        
            target_index = np.where(target_labels[: , 0] == newname[:-4])[0]
            target_index = int(target_index)
            target = int(target_labels[: , 1][target_index]) - 1
            targets.append(target)
          audio_read, sr = sf.read("/tmp/" + output_wav_file)
          audio_reads.append(audio_read)
          print(f"Reading file {output_wav_file} ...")
       
          
        X_arr, ts_list = openl3.get_audio_embedding(audio_reads, sr, batch_size=15 ,  hop_size=self.hop_size)

        X = tf.keras.preprocessing.sequence.pad_sequences(X_arr, maxlen=maxlen)
        X = np.asarray(X , dtype='float32')

        if i == 0:
          all_x = X
          all_x = np.asarray(all_x , dtype='float32')
        else: 
          all_x = np.concatenate((all_x ,  X ), axis=0)

        print(all_x.shape)

      for f in video_files:
        file_name = os.path.basename(f)
        with open(f"{self.output_folder}/audio-pickle/{file_name}-openl3.pkl", "wb") as f_out:
          pickle.dump(all_x[i], f_out)


      if self.label_path is not None:
        with open(f"{self.output_folder}/audio-pickle-all-X-openl3.pkl", "wb") as f_out:
          pickle.dump(all_x , f_out)

        targets = np.asarray(targets)
        with open(f"{self.output_folder}/audio-pickle-all-Y-openl3.pkl", "wb") as f_out:
          pickle.dump(targets , f_out)

      if self.output_file is not None:
            print(f"Starting to zip files to {self.output_file}")
            def zipdir(path, ziph):
                for root, dirs, files in os.walk(path):
                    folder = root[len(path):]
                    for file in files:
                        ziph.write(join(root, file), join(folder, file))

            zipf = zipfile.ZipFile(self.output_file, 'w', zipfile.ZIP_DEFLATED)
            zipdir(self.output_folder, zipf)
            zipf.close()
            print(f"Done zipping files to {self.output_file}")
        
      print("Done!")











`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
HOME_DIR = "drive/My Drive/Machine-Learning-Projects/"
audio_preprocessor = AudioPreprocessor(
    output_folder="train-tiny-audio", 
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny-audio.zip" ,
    video_folder= HOME_DIR + "cs231n-project/datasets/emotiw/train-tiny.zip",
    label_path = HOME_DIR + "cs231n-project/datasets/emotiw/Train_labels.txt"
)

X = audio_preprocessor.preprocess()

Video Preprocessor created with is_zip = True, video_folder = drive/My Drive/Machine-Learning-Projects/cs231n-project/datasets/emotiw/train-tiny.zip , output_folder = train-tiny-audio, output_file = drive/My Drive/Machine-Learning-Projects/cs231n-project/datasets/emotiw/train-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir train-tiny-audio_tmp...
Finished unzipping files

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i train-tiny-audio_tmp/101_12.mp4 -ab 3000k -ar 44100 /tmp/101_12.mp4extracted_audio.wav
... command successful.
Reading file 101_12.mp4extracted_audio.wav ...

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i train-tiny-audio_tmp/101_30.mp4 -ab 3000k -ar 44100 /tmp/101_30.mp4extracted_audio.wav
... command successful.
Reading file 101_30.mp4extracted_audio.wav ...

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i train-tiny-audio_tmp/108_13.mp4 -ab 3000k -ar 44100 /tmp/108_13.mp4extracted_audio.wav
... command successful.
Reading file 108_13.mp4

In [0]:
HOME_DIR = "drive/My Drive/Machine-Learning-Projects/"
audio_preprocessor = AudioPreprocessor(
    video_folder =  HOME_DIR + "cs231n-project/datasets/emotiw/val-tiny.zip",
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/val-tiny-audio.zip" ,
    output_folder="val-tiny-audio",
    label_path = HOME_DIR + "cs231n-project/datasets/emotiw/Val_labels.txt" )
audio_preprocessor.preprocess()

Video Preprocessor created with is_zip = True, video_folder = drive/My Drive/Machine-Learning-Projects/cs231n-project/datasets/emotiw/val-tiny.zip , output_folder = val-tiny-audio, output_file = drive/My Drive/Machine-Learning-Projects/cs231n-project/datasets/emotiw/val-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir val-tiny-audio_tmp...
Finished unzipping files

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i val-tiny-audio_tmp/115_10.mp4 -ab 3000k -ar 44100 /tmp/115_10.mp4extracted_audio.wav
... command successful.
Reading file 115_10.mp4extracted_audio.wav ...

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i val-tiny-audio_tmp/115_8.mp4 -ab 3000k -ar 44100 /tmp/115_8.mp4extracted_audio.wav
... command successful.
Reading file 115_8.mp4extracted_audio.wav ...

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i val-tiny-audio_tmp/115_9.mp4 -ab 3000k -ar 44100 /tmp/115_9.mp4extracted_audio.wav
... command successful.
Reading file 115_9.mp4extracted_audio.wav 

In [0]:
HOME_DIR = "drive/My Drive/Machine-Learning-Projects/"
TEST_LABELS_PATH = ""

audio_preprocessor = AudioPreprocessor(
    output_folder="test-tiny-audio", 
    output_file= HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny-audio.zip" ,
    video_folder= HOME_DIR + "cs231n-project/datasets/emotiw/test-tiny.zip",
    label_path = TEST_LABELS_PATH
)

X = audio_preprocessor.preprocess()